In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Credit Card customers**

**Predict Churning customers**

Problem :

A manager at the bank is disturbed with more and more customers leaving their credit card services. They would really appreciate if one could predict for them who is gonna get churned so they can proactively go to the customer to provide them better services and turn customers decisions in the opposite direction. We have only 16.07% of customers who have churned. Thus, it's a bit difficult to train our model to predict churning customers.

In [ ]:
# import library
import pandas as pd
import numpy as np

#matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
data = pd.DataFrame(data)
data.head()

# 1. Understanding the Data

> Based on the note from the data source, I decided to ignore the last 2 columns of the data, so the first thing I did was I had to delete it so it wouldn't interfere when analyzing the overall data

In [ ]:
#Deleting the last 2 columns
data.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                   'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
                   'CLIENTNUM'], 
          inplace=True, axis=1)
data

In [ ]:
data.dtypes

> Grouping columns by data type

In [ ]:
numerical_columns = ["Customer_Age", 
                     "Months_on_book", 
                     "Credit_Limit", 
                     "Total_Revolving_Bal", 
                     "Avg_Open_To_Buy", 
                     "Total_Amt_Chng_Q4_Q1", 
                     "Total_Trans_Amt", 
                     "Total_Trans_Ct", 
                     "Total_Ct_Chng_Q4_Q1", 
                     "Avg_Utilization_Ratio"]

categorical_columns = ['Attrition_Flag',
                       'Gender',
                       'Education_Level',
                       'Marital_Status',
                       'Income_Category',
                       'Card_Category']

discrete_columns = ['CLIENTNUM',
                    'Dependent_count',
                    'Total_Relationship_Count',
                    'Months_Inactive_12_mon',
                    'Contacts_Count_12_mon']

# **2. Identify the Data**

In [ ]:
data.info()

In [ ]:
print(data.duplicated().sum())
print(data.isnull().sum())

Because there are no null values and duplicated values found, it can be said that this data is clean, and can be continued for data analysis by looking at the distribution of the data.

In [ ]:
# Numerical Value 
fig, axes = plt.subplots(5,2, figsize=(15,16))
axes = [ax for axes_row in axes for ax in axes_row]

for i, col in enumerate(numerical_columns):
  plot = sns.histplot(data=data, x=col, ax=axes[i], fill=True)
plt.tight_layout()

In [ ]:
# Categorical Value
fig, axes = plt.subplots(3,2, figsize=(15,11))
axes = [ax for axes_row in axes for ax in axes_row]

for i, col in enumerate(categorical_columns):
  data[col].value_counts()[::-1].plot(kind='barh',ax=axes[i], title=col, fontsize=14)
  axes[i].set_ylabel('')
plt.tight_layout()

# **3. Analyze the Data**

> **Analyze the data better focus on attrition_Flag column to dig more information about churning customers**

> The first analysis is to look for outliers values, because these values must be determined to see the pattern of distribution so that we can study the data better.

> To find outliers, it is necessary to pay attention to using a boxplot chart

In [ ]:
fig, axes = plt.subplots(5,2, figsize=(10,20))
axes = [ax for axes_row in axes for ax in axes_row]

for i, col in enumerate(numerical_columns):
    plot = sns.boxplot(data=data, y=col, x='Attrition_Flag', ax=axes[i])
plt.tight_layout()

# **4. Data Pre-Processing**

# Label Eencoding for Target Value

> Then the analysis continues by seeing how the level of correlation between all columns, but the first do Label Encoder to change "Existing Customer" = 1, and "Attrited Customer" = 0

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

data['Attrition_Flag'] = label.fit_transform(data['Attrition_Flag'])
data.head()

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.heatmap(data.corr(), annot=True,cmap="YlGnBu")
cbar = ax.collections[0].colorbar
cbar.set_ticks([0, .25, .50, .75, 1])
cbar.set_ticklabels(['0%', '25%','50%', '75%', '100%'])
plt.show()

# One Hot Encoding for Categorical Value

Now we can know what column that has good corelation with "attrition flag" column

In [ ]:
#performing one-hot encoding
print('Before doing one-hot encoding',data.shape)
data_dumm = pd.get_dummies(data, prefix_sep='_')
print('After doing one-hot encoding',data_dumm.shape)
data_dumm.head()

# Removing Ouliers (STD Method)

In [ ]:
# Before Removing Outliers
data_dumm['Attrition_Flag'].value_counts()

In [ ]:
#Removing Outliers Function (Standard Deviation Method)
def remove_outliers(data_dumm, columns, tresh=3):
    dataset = data_dumm.copy()
    index_to_remove = []
    value_to_remove = []
    for col in columns:
        array_to_remove = dataset[col].values
        mean, std = np.mean(array_to_remove), np.std(array_to_remove)
        z_score = np.abs((array_to_remove - mean) / std)
        threshold = tresh
        good = z_score < threshold
        for i in range(len(good)):
            if good[i] == False:
                index_to_remove.append(i)
                value_to_remove.append(array_to_remove[i])
    index_to_remove = np.unique(np.array(index_to_remove))
    print("Removed rows: ", len(index_to_remove))
    dataset = dataset.drop(dataset.index[index_to_remove])
    return dataset

In [ ]:
data_after_removing_outliers = remove_outliers(data_dumm, numerical_columns)

In [ ]:
data_after_removing_outliers['Attrition_Flag'].value_counts()

In [ ]:
fig, axes = plt.subplots(5,2, figsize=(10,20))
axes = [ax for axes_row in axes for ax in axes_row]

for i, col in enumerate(numerical_columns):
    plot = sns.boxplot(data=data_after_removing_outliers, y=col, x='Attrition_Flag', ax=axes[i])
plt.tight_layout()

# Split Train and Test 

In [ ]:
X = data_after_removing_outliers.drop(['Attrition_Flag'],1)
y = data_after_removing_outliers['Attrition_Flag']

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 123, stratify=y)

# Oversampling (SMOTEENN)

In [ ]:
from imblearn.combine import SMOTEENN

smoteen = SMOTEENN()
X_train, y_train = smoteen.fit_resample(X_train, y_train)

# Standard Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train

# **5. Develop Model**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Cross K Validation
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, roc_auc_score, recall_score

def evaluate_model(model, X_test, y_test):
  y_pred = cross_val_predict(model, X_test, y_test, cv=5)
  cm = confusion_matrix(y_test, y_pred)
  print(cm)
  print("ROC AUC Score: ", roc_auc_score(y_test, y_pred))
  print("Accuracy Score: ", accuracy_score(y_test, y_pred))
  print("Precision Score: ", precision_score(y_test, y_pred))
  print("Recall Score: ", recall_score(y_test, y_pred))

## **K-Nearest Neighbor**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
evaluate_model(knn, X_test, y_test)

## **Support Vector Machine**

In [ ]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)

In [ ]:
evaluate_model(svm, X_test, y_test)

## **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
evaluate_model(lr, X_test, y_test)

## **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
evaluate_model(dt, X_test, y_test)

## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
evaluate_model(rf, X_test, y_test)

## **XG Boost**

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

In [ ]:
evaluate_model(xgb, X_test, y_test)

## **Light GBM**

In [ ]:
import lightgbm as lgb

gbm = lgb.LGBMClassifier()
gbm.fit(X_train, y_train)

In [ ]:
evaluate_model(gbm, X_test, y_test)

# **Hyperparameter Tuning for Light GBM**

In [ ]:
params = {
    'max_depth':[3,7,9,'max'],
    'n_estimators': [150, 200, 250],
    'num_leaves': [25, 35, 40],
    'learning_rate': [0.01, 0.22, 0.3]
    }

In [ ]:
gbm_grid = lgb.LGBMClassifier()

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator=gbm_grid,
             param_grid=params,
             scoring = 'roc_auc',
             n_jobs = -1,
             cv = 3,
             verbose=2
            )

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
import lightgbm as lgb

gbm_after_tuning = lgb.LGBMClassifier(learning_rate=0.22, max_depth=7, n_estimators=250, num_leaves=25)
gbm_after_tuning.fit(X_train, y_train)

In [ ]:
evaluate_model(gbm_after_tuning, X_test, y_test)

# **Hyperparameter Tuning For XG Boost**

In [ ]:
params_XGB = {
    'eta':[0.1, 0.3, 0.7],
    'max_depth': [3, 5, 6, 7],
    'gamma': [0, 10, 50]
    }

In [ ]:
xgb_grid = XGBClassifier()

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator=xgb_grid,
             param_grid=params_XGB,
             scoring = 'roc_auc',
             n_jobs = -1,
             cv = 3,
             verbose=2
            )

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
from xgboost import XGBClassifier

xgb_after_tuning = XGBClassifier(eta= 0.3, gamma= 0, max_depth= 5)
xgb_after_tuning.fit(X_train, y_train)

In [ ]:
evaluate_model(xgb_after_tuning, X_test, y_test)

# **Evaluation**

In [ ]:
# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')